In [2]:
import pandas as pd
import numpy as np

In [50]:
df = pd.read_csv("Data - Original Data.csv", header=1)

# Linearly interpolate population in Scotland
df['SCT_Working_Pop'] = df['SCT_Working_Pop'].interpolate()
df['rUK_Working_Pop'] = df['rUK_Working_Pop'].interpolate()
df['Year'] = df['Year'].fillna(method='ffill')
df = df.query("Year <= 2021")

defl = df["UK_Deflator (2023=100)"]

sct_pop = df["SCT_Working_Pop"]
rUK_pop = df["rUK_Working_Pop"]

for var in ['Consumption', 'Output', 'Compensation_of_Employees', 'Government_spending']:
    # Adjust for inflation using UK-wide deflator
    df[f'SCT_{var}'] = df[f'SCT_{var}'] / defl * 100
    df[f'UK_{var}'] = df[f'UK_{var}'] / defl * 100

for var in ['Consumption', 'Output', 'Compensation_of_Employees', 'Government_spending']:
    # rUK = UK - Scot
    df[f'rUK_{var}'] = df[f'UK_{var}'] - df[f'SCT_{var}']

# We use historic arithmetic mean as steady state ratio
g_y = np.mean(df['SCT_Government_spending'] / df['SCT_Output'])
g_y_ruk = np.mean(df['rUK_Government_spending'] / df['rUK_Output'])

print(f'Scotland: {int(min(df["Year"]))}Q1-{int(max(df["Year"]))}Q4 inflation-adjusted government spending-to-output ratio is {g_y}')
print(f'rUK: {int(min(df["Year"]))}Q1-{int(max(df["Year"]))}Q4 inflation-adjusted government spending-to-output ratio is {g_y_ruk}')

for var in ['Consumption', 'Output', 'Compensation_of_Employees', 'Government_spending']:
    # Express terms per capita
    df[f'SCT_{var}'] = df[f'SCT_{var}'] / sct_pop
    df[f'rUK_{var}'] = df[f'UK_{var}'] / rUK_pop

sct_pop_share = sum(df["SCT_Working_Pop"]) / (sum(df["rUK_Working_Pop"] + df["SCT_Working_Pop"]))

print(f"Scotland's share of UK's population in {int(min(df['Year']))}Q1-{int(max(df['Year']))}Q4 is: {sct_pop_share}")

# Map dataset variables to dynare endogenous variabes used for estimation
dynare_map = {
    'SCT_Output': 'y_obs',
    'SCT_Consumption': 'c_obs',
    'SCT_Compensation_of_Employees': 'w_obs',
    'SCT_Government_spending': 'g_obs',
    'rUK_Output': 'y_obs_ruk',
    'rUK_Consumption': 'c_obs_ruk',
    'rUK_Compensation_of_Employees': 'w_obs_ruk',
    'rUK_Government_spending': 'g_obs_ruk'
}

for var in ['Consumption', 'Output', 'Compensation_of_Employees', 'Government_spending']:
    for country in ['SCT', 'rUK']:
        # Log-transform and take first difference
        temp_diff_log = np.log(df[f'{country}_{var}']).diff()
        
        # De-mean
        df[dynare_map[f'{country}_{var}']] = temp_diff_log[1:] - np.average(temp_diff_log[1:])

df


Scotland: 1998Q1-2021Q4 inflation-adjusted government spending-to-output ratio is 0.2563136588315356
rUK: 1998Q1-2021Q4 inflation-adjusted government spending-to-output ratio is 0.1796672623493271
Scotland's share of UK's population in 1998Q1-2021Q4 is: 0.0851089237635477


,Year,Quarter,SCT_Deflator (2018=100),SCT_Output,SCT_Consumption,SCT_Compensation_of_Employees,SCT_Working_Pop,UK_Deflator (2023=100),UK_Output,UK_Consumption,...,rUK_Compensation_of_Employees,rUK_Government_spending,c_obs,c_obs_ruk,y_obs,y_obs_ruk,w_obs,w_obs_ruk,g_obs,g_obs_ruk
0,1998.0,1.0,65.1,0.009646,0.006226,0.004697,3281902.0,61.2685,401149.040698,256489.060447,...,0.005364,0.001852,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1998.0,2.0,65.4,0.009681,0.006257,0.004800,3281797.0,61.5281,403284.027948,258408.759575,...,0.005446,0.001796,0.002888,0.004139,0.000865,0.001119,0.018319,0.011460,-0.020189,-0.036180
2,1998.0,3.0,65.3,0.009552,0.006294,0.004842,3281692.0,62.1079,404497.656498,261387.037720,...,0.005488,0.001804,0.003753,0.008144,-0.016245,-0.001182,0.005345,0.003829,0.005185,-0.001311
3,1998.0,4.0,65.1,0.009483,0.006291,0.004841,3281587.0,62.4503,407027.668402,260703.311273,...,0.005481,0.001842,-0.002549,-0.005932,-0.009959,0.002050,-0.003617,-0.005024,-0.013417,0.015081
4,1999.0,1.0,65.5,0.009556,0.006370,0.004901,3281482.0,62.5442,409948.484432,264598.475958,...,0.005619,0.001863,0.010489,0.011519,0.004850,0.002968,0.008938,0.021026,0.011340,0.005830
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,2020.0,4.0,108.2,0.011315,0.006748,0.006242,3494172.0,99.4575,542609.657391,307484.101249,...,0.007296,0.002850,0.004204,-0.017579,0.013538,0.007248,0.022592,0.021212,0.011051,0.010988
92,2021.0,1.0,109.8,0.011246,0.006585,0.006198,3494517.0,100.1883,536896.024785,298013.839939,...,0.007229,0.002810,-0.026467,-0.035225,-0.008895,-0.015398,-0.010411,-0.013011,0.022359,-0.019705
93,2021.0,2.0,108.0,0.012079,0.007480,0.006447,3494517.0,98.3042,571947.078558,335552.295833,...,0.007571,0.003190,0.125378,0.116821,0.068606,0.060553,0.036014,0.042391,0.022457,0.121110
94,2021.0,3.0,108.5,0.012279,0.007537,0.006470,3494517.0,99.2256,581861.938854,344771.913700,...,0.007653,0.003158,0.005475,0.025288,0.013616,0.014498,0.000184,0.006983,-0.003241,-0.015631


In [38]:
# Export for dynare

dynare_sct_data_cols = ["y_obs", "c_obs", "w_obs", "g_obs"]
dynare_ruk_data_cols = ["y_obs_ruk", "c_obs_ruk", "w_obs_ruk", "g_obs_ruk"]
dynare_data = df[dynare_sct_data_cols + dynare_ruk_data_cols][1:]
dynare_data.to_csv('../DynareData.csv', index=False)

dynare_data

,y_obs,c_obs,w_obs,g_obs,y_obs_ruk,c_obs_ruk,w_obs_ruk,g_obs_ruk
1,0.000865,0.002888,0.018319,-0.020189,0.001119,0.004139,0.011460,-0.036180
2,-0.016245,0.003753,0.005345,0.005185,-0.001182,0.008144,0.003829,-0.001311
3,-0.009959,-0.002549,-0.003617,-0.013417,0.002050,-0.005932,-0.005024,0.015081
4,0.004850,0.010489,0.008938,0.011340,0.002968,0.011519,0.021026,0.005830
5,-0.006969,0.012236,0.022420,-0.006268,-0.000311,0.005814,0.016149,-0.014131
...,...,...,...,...,...,...,...,...
91,0.013538,0.004204,0.022592,0.011051,0.007248,-0.017579,0.021212,0.010988
92,-0.008895,-0.026467,-0.010411,0.022359,-0.015398,-0.035225,-0.013011,-0.019705
93,0.068606,0.125378,0.036014,0.022457,0.060553,0.116821,0.042391,0.121110
94,0.013616,0.005475,0.000184,-0.003241,0.014498,0.025288,0.006983,-0.015631


In [5]:
import scipy.io as sio
from datetime import date

# Two governments
# Extract MH draws and store them as CSV. 
mat = sio.loadmat("/Users/edvardasvabuolas/Documents/University/Dissertation/dissertation/metropolis/dissertation_mh1_blck1.mat")

mat_df = pd.DataFrame(mat['x2'])
mat_df = mat_df.iloc[:, 4:10]
mat_df.columns = ['phi_b', 'phi_g', 'phi_b_f', 'phi_g_f', 'rho_g', 'rho_g_f']
mat_df.to_csv(f"/Users/edvardasvabuolas/Documents/University/Dissertation/Output/MCMC_SCOT_RUK_{date.today()}.csv")
mat_df

,phi_b,phi_g,phi_b_f,phi_g_f,rho_g,rho_g_f
0,0.236860,0.075762,0.236873,0.075770,0.924251,0.924251
1,0.236860,0.075762,0.236873,0.075770,0.924251,0.924251
2,0.236860,0.075762,0.236873,0.075770,0.924251,0.924251
3,0.315313,0.064457,0.388701,0.089373,0.909613,0.938142
4,0.315313,0.064457,0.388701,0.089373,0.909613,0.938142
...,...,...,...,...,...,...
19995,0.225729,0.107164,0.378064,0.066311,0.835557,0.863133
19996,0.225729,0.107164,0.378064,0.066311,0.835557,0.863133
19997,0.225729,0.107164,0.378064,0.066311,0.835557,0.863133
19998,0.225729,0.107164,0.378064,0.066311,0.835557,0.863133


In [3]:
import scipy.io as sio
from datetime import date

# One governments
# Extract MH draws and store them as CSV. 
mat = sio.loadmat("/Users/edvardasvabuolas/Documents/University/Dissertation/dissertation/metropolis/dissertation_mh1_blck1.mat")

mat_df = pd.DataFrame(mat['x2'])
mat_df = mat_df.iloc[:, 2:5]
mat_df.columns = ['phi_b_uk', 'phi_g_uk', 'rho_g_uk']
mat_df.to_csv(f"/Users/edvardasvabuolas/Documents/University/Dissertation/Output/MCMC_UK_{date.today()}.csv")
mat_df

,phi_b_uk,phi_g_uk,rho_g_uk
0,0.004656,0.006227,0.195484
1,0.004656,0.006227,0.195484
2,0.004656,0.006227,0.195484
3,0.004462,0.005090,0.311291
4,0.004462,0.005090,0.311291
...,...,...,...
19995,0.005380,0.004757,0.281375
19996,0.005380,0.004757,0.281375
19997,0.005638,0.006344,0.391866
19998,0.005638,0.006344,0.391866


In [132]:
import scipy.io as sio
mat = sio.loadmat("/Users/edvardasvabuolas/Documents/University/Dissertation/dissertation/Output/dissertation_results.mat")['oo_']['irfs']
mat_iter = mat[0][0][0][0]

df = pd.DataFrame()
for var in range(0,len(mat_iter)):
    df[var] = mat_iter[var][0]
df.columns = mat[0][0].dtype.names
df

,debt_eps_g,debt_f_eps_g,b_eps_g,b_f_eps_g,r_real_eps_g,r_real_f_eps_g,g_eps_g,g_f_eps_g,tr_eps_g,tr_f_eps_g,tau_eps_g,tau_f_eps_g,c_eps_g,c_f_eps_g,y_eps_g,y_f_eps_g,wp_eps_g,wp_f_eps_g,n_eps_g,n_f_eps_g
0,0.273601,-5.177624e-17,0.017017,3.173946e-16,0.008539,-1.460399e-17,0.250000,0.000000e+00,-0.023601,5.177624e-17,-0.088142,1.359468e-16,-0.250740,4.532867e-17,-0.000747,4.531587e-17,-0.379139,5.895325e-16,-0.000996,5.688149e-17
1,0.233666,-9.654587e-17,0.238160,1.101437e-15,0.009837,3.130801e-17,0.217500,-2.641846e-18,-0.016166,9.390403e-17,-0.037565,4.068087e-16,-0.218802,4.341137e-17,-0.001308,4.072086e-17,-0.281438,9.586087e-16,-0.001744,5.133231e-17
2,0.199103,-1.090922e-16,0.437529,2.212325e-15,0.008712,5.930678e-17,0.189225,6.260782e-19,-0.009878,1.097182e-16,-0.005485,4.667297e-16,-0.188608,4.495759e-17,0.000612,4.525080e-17,-0.191332,1.098313e-15,0.000816,5.942971e-17
3,0.169160,-1.274695e-16,0.615867,3.648805e-15,0.007534,8.232623e-17,0.164626,7.568996e-18,-0.004534,1.350384e-16,0.020718,5.734978e-16,-0.162186,4.634169e-17,0.002436,5.332287e-17,-0.113926,1.334628e-15,0.003247,7.200532e-17
4,0.143239,-1.514997e-16,0.774564,5.406053e-15,0.006489,1.022134e-16,0.143224,1.674520e-17,-0.000015,1.682449e-16,0.042627,7.206218e-16,-0.139243,4.798160e-17,0.003977,6.390876e-17,-0.048221,1.650266e-15,0.005303,8.772246e-17
5,0.120828,-1.810551e-16,0.914941,7.472434e-15,0.005574,1.193107e-16,0.124605,2.705726e-17,0.003778,2.081124e-16,0.060815,9.015079e-16,-0.119350,5.062329e-17,0.005253,7.665492e-17,0.007136,2.033417e-15,0.007003,1.061285e-16
6,0.101478,-2.158723e-16,1.038214,9.830655e-15,0.004774,1.338047e-16,0.108407,3.766400e-17,0.006929,2.535363e-16,0.075718,1.110215e-15,-0.102114,5.480703e-17,0.006290,9.125873e-17,0.053374,2.473430e-15,0.008387,1.268359e-16
7,0.084799,-2.555515e-16,1.145500,1.245900e-14,0.004075,1.458718e-16,0.094314,4.793166e-17,0.009515,3.034832e-16,0.087718,1.341395e-15,-0.087193,6.085025e-17,0.007118,1.074024e-16,0.091584,2.960185e-15,0.009491,1.494351e-16
8,0.070450,-2.995798e-16,1.237831,1.533228e-14,0.003465,1.556868e-16,0.082053,5.739524e-17,0.011603,3.569750e-16,0.097151,1.590137e-15,-0.074291,6.889701e-17,0.007761,1.247634e-16,0.122727,3.483993e-15,0.010348,1.735093e-16
9,0.058133,-3.473574e-16,1.316170,1.842266e-14,0.002932,1.634192e-16,0.071386,6.572640e-17,0.013253,4.130838e-16,0.104317,1.851868e-15,-0.063146,7.896129e-17,0.008239,1.430264e-16,0.147652,4.035593e-15,0.010986,1.986495e-16
